In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('dataset/cleaned_combined_generation_dataset.csv')
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset.head()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,electricity_demand,dwpt,wspd,wdir,prcp,rhum,pres,season,month,population
0,2000-01-01 00:00:00,5,0,1,17.483571,0,512.319382,16.036364,51.8,20.0,0.0,50.045455,1010.0,1,1,1.569190e+07
1,2000-01-01 01:00:00,5,1,1,14.308678,0,439.055673,16.036364,51.8,20.0,0.0,50.045455,1010.0,1,1,1.569198e+07
2,2000-01-01 02:00:00,5,2,1,18.238443,0,539.090652,16.036364,51.8,20.0,0.0,50.045455,1010.0,1,1,1.569206e+07
3,2000-01-01 03:00:00,5,3,1,22.615149,0,523.248968,16.036364,51.8,20.0,0.0,50.045455,1010.0,1,1,1.569215e+07
4,2000-01-01 04:00:00,5,4,1,13.829233,0,516.822822,16.036364,51.8,20.0,0.0,50.045455,1010.0,1,1,1.569223e+07


In [3]:
# hourly_population=pd.read_csv('dataset/hourly_population.csv')

In [4]:
dataset.dtypes

timestamp             datetime64[ns]
day_of_week                    int64
hour_of_day                    int64
is_weekend                     int64
temperature                  float64
is_holiday                     int64
electricity_demand           float64
dwpt                         float64
wspd                         float64
wdir                         float64
prcp                         float64
rhum                         float64
pres                         float64
season                         int64
month                          int64
population                   float64
dtype: object

In [5]:
dataset.drop(['timestamp'], axis=1, inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)
val_data_scaled = scaler.transform(val_data)
test_data_scaled = scaler.transform(test_data)

In [8]:
# CNN Features
cnn_features = ["temperature", "dwpt", "wspd", "wdir", "prcp", "rhum", "pres"]
X_train_cnn = train_data[cnn_features]
X_val_cnn = val_data[cnn_features]
X_test_cnn = test_data[cnn_features]

# LSTM Features
lstm_features = ["day_of_week", "hour_of_day", "season", "month"]
X_train_lstm = train_data[lstm_features]
X_val_lstm = val_data[lstm_features]
X_test_lstm = test_data[lstm_features]

# ANN Features
ann_features = ["is_holiday", "is_weekend", "population"]
X_train_ann = train_data[ann_features]
X_val_ann = val_data[ann_features]
X_test_ann = test_data[ann_features]

# Target Variable
y_train = train_data["electricity_demand"]
y_val = val_data["electricity_demand"]
y_test = test_data["electricity_demand"]

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Flatten

X_train_cnn = X_train_cnn.values.reshape((X_train_cnn.shape[0], X_train_cnn.shape[1], 1))
X_val_cnn = X_val_cnn.values.reshape((X_val_cnn.shape[0], X_val_cnn.shape[1], 1))
X_test_cnn = X_test_cnn.values.reshape((X_test_cnn.shape[0], X_test_cnn.shape[1], 1))


In [10]:
# Build and train the CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    Flatten(),
    Dense(24, activation='linear')
])
cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_val_cnn, y_val), epochs=10)


Epoch 1/10
4208/4208 [==============================] - 32s 6ms/step - loss: 9087.9326 - val_loss: 7123.6890
Epoch 2/10
4208/4208 [==============================] - 19s 4ms/step - loss: 7152.6968 - val_loss: 7253.6221
Epoch 3/10
4208/4208 [==============================] - 15s 4ms/step - loss: 7135.7837 - val_loss: 7398.7661
Epoch 4/10
4208/4208 [==============================] - 15s 4ms/step - loss: 7124.3174 - val_loss: 7552.0249
Epoch 5/10
4208/4208 [==============================] - 14s 3ms/step - loss: 7123.9658 - val_loss: 7169.9204
Epoch 6/10
4208/4208 [==============================] - 14s 3ms/step - loss: 7116.8848 - val_loss: 7339.6875
Epoch 7/10
4208/4208 [==============================] - 14s 3ms/step - loss: 7125.4014 - val_loss: 7056.6846
Epoch 8/10
4208/4208 [==============================] - 14s 3ms/step - loss: 7106.5547 - val_loss: 7061.4458
Epoch 9/10
4208/4208 [==============================] - 14s 3ms/step - loss: 7110.5396 - val_loss: 7165.5493
Epoch 10/10
4208/42

In [11]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

y_test_reshaped = y_test.values.reshape(-1, 24)
# Make predictions with the CNN model
cnn_output_test = cnn_model.predict(X_test_cnn)

# Calculate metrics for CNN model
rmse_cnn = root_mean_squared_error(y_test, cnn_output_test)
# rmse_cnn = np.sqrt(mse_cnn)
mae_cnn = mean_absolute_error(y_test, cnn_output_test)
r2_cnn = r2_score(y_test, cnn_output_test)

print(f'CNN Model - MSE: {rmse_cnn}, RMSE: {rmse_cnn}, MAE: {mae_cnn}, R²: {r2_cnn}')

ValueError: cannot reshape array of size 42073 into shape (24)